# Richter's Predictor: Modeling Earthquake Damage
## MMAI 869 - Team College

URL: https://www.drivendata.org/competitions/57/nepal-earthquake/

Hi Team.  Let's use this notebook to record our work for the ML challenge.

Please make a copy of this notebook and try building your models.  Once you have completed, please copy your code and post it as a section (one # in markdown) in this notebook.

Rememmber to append the name of your model the list **models** and the predicted target of the test data in the dataframe **benchmark**.

The last section of this notebook will show the score of the models and we can review each other's attempt and improve our models.

I have created 2 sample models for your reference.

You may condense some sections for easy viewing.

# Preparation

### Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier



### Import Data files

In [3]:
train_values = pd.read_csv("train_values.csv")
train_values.set_index('building_id',inplace=True)
train_labels = pd.read_csv("train_labels.csv")
train_labels.set_index('building_id',inplace=True)
test_values = pd.read_csv("test_values.csv")
test_values.set_index('building_id',inplace=True)

train = train_values.join(train_labels)


In [4]:
all_columns = list(train.columns)
x_columns = list(train.columns[:-1])  

pdx = train[x_columns]
pdy = train['damage_grade']

# Let's use 869 as the common random seed
# split data into 70/30 training to testing
x_train,x_test,y_train,y_test = train_test_split(pdx,pdy,train_size = 0.7,random_state=869)

models = ['Perfect'] # Keep a dictionary of models we tried
benchmark = pd.DataFrame(y_test)

benchmark.columns = models # Keep a copy of the predicted y of the test data fro benchmarking

# x_columns has all names of all features
# x_train are values of the features for training data
# y_train are targets for training data
# x_test are values of the features for testing data
# y_test are targets for the testing data

In [4]:
# the scoring method for evaluation
scoring = "f1_micro"


### explore data -Jing

In [5]:
train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [6]:
# check null value, no null, then only need covert category data to number
train.isna().sum()

geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo                 0
has_superstructure_rc_non_engine

In [7]:
# check datatype of each feature
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          260601 non-null  int64 
 1   geo_level_2_id                          260601 non-null  int64 
 2   geo_level_3_id                          260601 non-null  int64 
 3   count_floors_pre_eq                     260601 non-null  int64 
 4   age                                     260601 non-null  int64 
 5   area_percentage                         260601 non-null  int64 
 6   height_percentage                       260601 non-null  int64 
 7   land_surface_condition                  260601 non-null  object
 8   foundation_type                         260601 non-null  object
 9   roof_type                               260601 non-null  object
 10  ground_floor_type                       260601 non-

In [8]:
train.count()

geo_level_1_id                            260601
geo_level_2_id                            260601
geo_level_3_id                            260601
count_floors_pre_eq                       260601
age                                       260601
area_percentage                           260601
height_percentage                         260601
land_surface_condition                    260601
foundation_type                           260601
roof_type                                 260601
ground_floor_type                         260601
other_floor_type                          260601
position                                  260601
plan_configuration                        260601
has_superstructure_adobe_mud              260601
has_superstructure_mud_mortar_stone       260601
has_superstructure_stone_flag             260601
has_superstructure_cement_mortar_stone    260601
has_superstructure_mud_mortar_brick       260601
has_superstructure_cement_mortar_brick    260601
has_superstructure_t

In [9]:
# check the output to see whether balanced
train["damage_grade"].value_counts()


2    148259
3     87218
1     25124
Name: damage_grade, dtype: int64

2: 148259/260601 = 0.56
3: 87218/260601 = 0.33
1: 25124/260601 = 0.096

# they are not balanced, we need think about balance them later
# which take the 
# forest resembling 
# the model should be used are 

*	Logistic Regression
*   K-Nearest Neighbors
*	Support Vector Machines
*	Decision Tree Classifiers/Random Forests
*	Naive Bayes
*	Linear Discriminant Analysis
*   quadratic Discriminant Analysis


names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]




### convert categorical data into number using onehotencoder since get_dummie migth get unbalanced category -- Jing, Nov 12

In [5]:
# get category columns
cat_columns = train.select_dtypes(include=['object']).columns

In [7]:
# don't use get_dummy, instead use oneHotEncoder, train first then the same one will use for training and finial test
# use columnTransformer to applies transformers to columns of an array or pandas DataFrame

ct = ColumnTransformer([("one-hot-encoder", OneHotEncoder(), cat_columns)], remainder ="passthrough")

# should fit the column transfer first
ct.fit(x_train)



ColumnTransformer(remainder='passthrough',
                  transformers=[('one-hot-encoder', OneHotEncoder(),
                                 Index(['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status'],
      dtype='object'))])

# define help functions to train data and register predict value

In [8]:
# define a function to do the steps:
# can use some global variables, do not need define everything
def train_model( X, y, model, model_name, param_grid, scoring, data_prep, cv=5 ):
    # define pipeline
    pipeline = Pipeline(steps=[("data_prep", data_prep),
                              ("model", model)])
    
    # use gridSearchCV to get the best model
    
    gs = GridSearchCV(pipeline,
                    param_grid=param_grid,
                    scoring=scoring,
                    cv=cv)
    
    #fit model
    gs.fit(X, y)
        
    #print the best model score
    print("{} training data average f1-score: {}".format(model_name, gs.score(X, y)))
    
    #return the trained model in case needed
    return gs  





In [9]:
# define the function to predict data and put into benchmark

def pred_target(model_name, model, X):
    y_pred = model.predict(X)
    # Register model and save result for comparison
    models.append(model_name)
    benchmark[model_name] = y_pred
    return 
    
    
    
    
    

In [10]:
# define logistic regression but for multiple class
model_name = "MLR"
model = LogisticRegression(multi_class="multinomial", solver='lbfgs')


In [11]:
# define parameters
params = [{}]

In [12]:
# get gs_lr

gs = train_model(x_train, y_train, model, model_name, params, scoring, ct)


C:\Users\billc\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\billc\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

MLR training data average f1-score: 0.5694660673171801


In [13]:
# predict the value and put into benchmark

pred_target(model_name, gs, x_test)

In [10]:
# build separate model
# build model for KNN
model_name = "KNN"
model = KNeighborsClassifier()

k_range = list(range(1, 10))
params = dict(model__n_neighbors=k_range)


In [ ]:
# get gs_knn

gs = train_model(x_train, y_train, model, model_name, params, scoring, ct)


In [32]:
# predict the value and put into benchmark

pred_target(model_name, gs, x_test)

In [ ]:
#  LDA




# Null Model that Assigns 2 as the Prediction - Kenny, Nov 11

### Feature Engineering

In [18]:
# Code for feature engineering

In [7]:
train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [9]:
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          260601 non-null  int64 
 1   geo_level_2_id                          260601 non-null  int64 
 2   geo_level_3_id                          260601 non-null  int64 
 3   count_floors_pre_eq                     260601 non-null  int64 
 4   age                                     260601 non-null  int64 
 5   area_percentage                         260601 non-null  int64 
 6   height_percentage                       260601 non-null  int64 
 7   land_surface_condition                  260601 non-null  object
 8   foundation_type                         260601 non-null  object
 9   roof_type                               260601 non-null  object
 10  ground_floor_type                       260601 non-

### Modelling

In [19]:
# Code for modelling and tuning with cross-validation
# scoring='f1_micro' can be used in cross_val_score function for tuning with respect to f1 micro average

# Please append the predicted y of the test data into the dataframne benchmark and name of the model into models
models.append('All 2')
benchmark['All 2']=2

# Plain Logistic Regression - Kenny, Nov 11

### Faeture Engineering - Plain Logictic Regression

In [41]:
x_train_dummies = pd.get_dummies(x_train).iloc[:,3:]  # transform categorical to dummy and ignore geo_id
x_test_dummies = pd.get_dummies(x_test).iloc[:,3:]  # transform categorical to dummy and ignore geo_id


ValueError: x array after removing special codes and missing values must be numerical.

In [39]:
x_train[var]

building_id
672415     7
863447    26
241545    27
723642    11
219465     8
          ..
987423     2
165222    17
125223     7
656036    23
402311     4
Name: geo_level_1_id, Length: 182420, dtype: int64

### Modelling - Plain Logistic Regression

In [27]:
# import library
from sklearn.linear_model import LogisticRegression

# fit the model
model_LR = LogisticRegression(max_iter=100000)
model_LR.fit(x_train_dummies, y_train)

# predict on test set
y_pred = model_LR.predict(x_test_dummies)

# Register model and save result for comparison
models.append('Plain LR')
benchmark['Plain LR'] = y_pred

# Your Modelling

In [22]:
# x_columns has all names of all features
# x_train are values of the features for training data
# y_train are targets for training data
# x_test are values of the features for testing data
# y_test are targets for the testing data

# Need to import library in your code
# scoring='f1_micro' can be used in cross_val_score function for tuning with respect to f1 micro average

# Please append the predicted y of the test data into the dataframne benchmark and name of the model into models

# You may refer to the 2 examples above

# Performance Evaluation (Micro Averaged F1 Score) using test 

In [ ]:
results = []

In [23]:
for i, model_name in enumerate(models):
    score = f1_score(y_test, benchmark[model_name], average='micro')
    result.append(score)
    print(model_name,"- Micro Average F1 = ",
            score)

Perfect - Micro Average F1 =  1.0
All 2 - Micro Average F1 =  0.567439659252248
Plain LR - Micro Average F1 =  0.5860119466366508


In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitile("algorithm comparison")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(models)
plt.show()




# once we choose the model, we should use the model train all the data then predict the final result

In [ ]:
# use the choosen model train all the training data



In [ ]:
# choosen model

In [10]:
# build separate model
# build model for KNN
finanl_model_name = "KNN"
final_model = KNeighborsClassifier()

k_range = list(range(1, 10))
params = dict(model__n_neighbors=k_range)


In [ ]:
# get gs_knn

final_gs = train_model(pdx, pdy, final_model, final_model_name, params, scoring, ct)


In [21]:
# consider add weight to the imbanlanced data
# do it in random forest

In [22]:
# predict the final data

pred_result = final_gs.predict(test_value)


In [23]:
# output the data file as required format
test_value_new = test_value.copy(deep=True)
test_value_new["damage_grade"] = pred_result
output = test_value_new['damage_grade']
output.to_csv("college_team_earthquake_first_submit.csv")

